In [9]:
import csv
import random
import requests
from sentence_transformers import SentenceTransformer, util

# Define the API Key and endpoint
API_KEY = "6d1fda2210ea41c69edf6a9ce33f4b13"
NEWS_API_URL = "https://newsapi.org/v2/everything"

# Define the search parameters
query = "supply chain disruption"
params = {
    "q": "supply chain disruption",
    "apiKey": API_KEY,
    "pageSize": 100,
    "sortBy": "relevance",
}

# Fetch news articles
response = requests.get(NEWS_API_URL, params=params)
articles = response.json().get("articles", [])

# Define the risk factors
risk_factors = [
    "Logistical",
    "Natural Disasters",
    "Market",
    "Regulatory",
    "Economic",
    "Geopolitical",
    "Customs Delays",
    "Supply chain disruptions",
    "Global competition",
    "Rising labor costs",
]

# Load a pre-trained model from Hugging Face
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode the risk factors
risk_factor_embeddings = model.encode(risk_factors, convert_to_tensor=True)

# Process articles and calculate risk factor scores
examples = []
for article in articles[:200]:  # Limit to 200 articles
    title = article.get("title", "No Title")
    description = article.get("description", "No Description")
    example_text = f"{title}: {description}"

    # Encode the example text
    example_embedding = model.encode(example_text, convert_to_tensor=True)

    # Calculate similarity scores with risk factors
    similarities = util.cos_sim(example_embedding, risk_factor_embeddings)[0]

    # Convert similarity scores to a list of decimal values
    risk_scores = similarities.tolist()

    # Determine the final risk factor and its value (highest similarity score)
    max_score_index = risk_scores.index(max(risk_scores))
    final_risk_factor = risk_factors[max_score_index]
    final_risk_value = max(risk_scores)

    # Append the example with risk scores, final risk factor, and its value
    examples.append((example_text, risk_scores, final_risk_factor, final_risk_value))

# Write to CSV file
csv_file_path = "news_import_export_risks_file.csv"
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Example"] + risk_factors + ["Final Risk Factor", "Final Risk Value"])
    # Write rows with risk scores, final risk factor, and its value
    for example, scores, final_risk_factor, final_risk_value in examples:
        writer.writerow([example] + scores + [final_risk_factor, final_risk_value])

print(f"CSV file '{csv_file_path}' created successfully with {len(examples)} rows.")



# Load a pre-trained model from Hugging Face
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encode the risk factors
risk_factor_embeddings = model.encode(risk_factors, convert_to_tensor=True)

# Process articles and calculate risk factor scores
examples = []
for article in articles[:200]:  # Limit to 200 articles
    title = article.get("title", "No Title")
    description = article.get("description", "No Description")
    example_text = f"{title}: {description}"

    # Encode the example text
    example_embedding = model.encode(example_text, convert_to_tensor=True)

    # Calculate similarity scores with risk factors
    similarities = util.cos_sim(example_embedding, risk_factor_embeddings)[0]

    # Convert similarity scores to a list of decimal values
    risk_scores = similarities.tolist()

    # Determine the final risk value (highest similarity score)
    final_risk_value = max(risk_scores)

    # Append the example with risk scores and the final risk value
    examples.append((example_text, risk_scores, final_risk_value))

# Write to CSV file
csv_file_path = "news_import_export_risks_file.csv"
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Example"] + risk_factors + ["Final Risk Value"])
    # Write rows with risk scores and the final risk value
    for example, scores, final_risk_value in examples:
        writer.writerow([example] + scores + [final_risk_value])

print(f"CSV file '{csv_file_path}' created successfully with {len(examples)} rows.")


CSV file 'news_import_export_risks_file.csv' created successfully with 100 rows.


In [10]:
import pandas as pd

# Path to the CSV file
csv_file_path = "/content/news_import_export_risks_file.csv"

# Load and display the CSV file
df = pd.read_csv(csv_file_path)
print(df.head())  # Display the first 5 rows


                                             Example  Logistical  \
0  The Most Dangerous People on the Internet in 2...    0.136250   
1  Geopolitics and AI will affect the chip indust...    0.173755   
2  Moselle Lock Closure: How To Mitigate Supply C...    0.199159   
3  NATO is working to reroute data through space,...    0.185938   
4  The FDA says the shortage of popular weight lo...    0.123055   

   Natural Disasters    Market  Regulatory  Economic  Geopolitical  \
0           0.312669  0.104376    0.087930  0.113044      0.212882   
1           0.135852  0.234857    0.213022  0.325603      0.426412   
2           0.230963  0.149327    0.195710  0.199763      0.077721   
3           0.146406  0.072456    0.053196  0.070176      0.319405   
4          -0.046860  0.149906    0.127936  0.119104      0.047725   

   Customs Delays  Supply chain disruptions  Global competition  \
0        0.054294                  0.208997            0.218297   
1        0.085301                  0

In [ ]:
import csv
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk

# Download the VADER lexicon if not already done
nltk.download("vader_lexicon")

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Perform sentiment analysis
sentiment_data = []
for article in articles[:200]:  # Limit to 200 articles
    title = article.get("title", "No Title")
    description = article.get("description", "No Description")
    text = f"{title}: {description}"

    # Analyze sentiment
    sentiment_scores = sia.polarity_scores(text)
    sentiment_category = (
        "Positive"
        if sentiment_scores["compound"] > 0.05
        else "Negative"
        if sentiment_scores["compound"] < -0.05
        else "Neutral"
    )

    # Append sentiment data
    sentiment_data.append((text, sentiment_scores, sentiment_category))

# Write sentiment data to a CSV file
csv_file_path = "news_sentiment_analysis.csv"
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    # Write header
    writer.writerow(["Text", "Positive", "Neutral", "Negative", "Compound", "Category"])
    # Write rows with sentiment scores and categories
    for text, scores, category in sentiment_data:
        writer.writerow([text, scores["pos"], scores["neu"], scores["neg"], scores["compound"], category])

print(f"CSV file '{csv_file_path}' created successfully with {len(sentiment_data)} rows.")
